In [1]:
import pandas as pd
import numpy as np
from numpy import asarray

from PIL import Image
from mtcnn.mtcnn import MTCNN






#from numpy import load
#from numpy import expand_dims
#from numpy import savez_compressed

#import cv2

#from keras.models import load_model





#import os.path
#from os import path
#from os import listdir
#from os.path import isdir

#from tqdm import tqdm_notebook

#import shutil
#import os, shutil

#import timeit



import matplotlib.pyplot as plt
%matplotlib inline



import warnings
warnings.filterwarnings('ignore')

## The Main DataFrames:

In [2]:
#Load the converted metadata files for the IMDB and Wikipedia images
imdb = pd.read_csv('photo_metadata.csv')
wiki = pd.read_csv('wiki_photo_metadata.csv')

#Concat the two pandas DataFrames
photo_df = pd.concat([imdb, wiki]).reset_index(drop=True)

## The metadata mainframe information


### Given:
-**name:** Name of the person

-**dob:** date of birth (Matlab serial date number)

-**gender:** 0 for female and 1 for male, NaN if unknown

-**photo_taken:** year when the photo was taken

-**full_path:** path to file

-**face_location:** location of the face. To crop the face in Matlab run img(face_location(2):face_location(4),face_location(1):face_location(3),:))

-**face_score:** detector score (the higher the better). Inf implies that no face was found in the image and the face_location then just returns the entire image

**Not Seen In This Notebook:**

-**second_face_score:** detector score of the face with the second highest score. This is useful to ignore images with more than one face. second_face_score is NaN if no second face was detected.**Not Seen In This Notebook**

-**celeb_names (IMDB only):** list of all celebrity names

-**celeb_id (IMDB only):** index of celebrity name


### Created:

-**age_when_taken** birthdate subtracted from photo_taken

In [3]:
photo_df.head()

,name,dob,gender,photo_taken,age_when_taken,file_path,face_location,face_score
0,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm124825600_...,[1072.926 161.838 1214.784 303.696],1.459693
1,Fred Astaire,1899-05-10,1.0,1970,71,image_data/imdb_data/01/nm0000001_rm3343756032...,[477.184 100.352 622.592 245.76 ],2.543198
2,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm577153792_...,[114.96964309 114.96964309 451.68657236 451.68...,3.455579
3,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm946909184_...,[622.88550564 424.21750384 844.33900767 645.67...,1.872117
4,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm980463616_...,[1013.85900236 233.88204221 1201.5861278 42...,1.158766


In [4]:
#There are 511817 total images across 73399 individuals
photo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384199 entries, 0 to 384198
Data columns (total 8 columns):
name              384199 non-null object
dob               384199 non-null object
gender            384199 non-null float64
photo_taken       384199 non-null int64
age_when_taken    384199 non-null int64
file_path         384199 non-null object
face_location     384199 non-null object
face_score        384199 non-null float64
dtypes: float64(2), int64(2), object(4)
memory usage: 23.4+ MB


In [5]:
def resize_picture(filename, dimensions = (160,160), margin = 0):
            
    # load the image
    image = Image.open(filename)
    image = image.convert('RGB')
    image_array = asarray(image)
    
    #Set a margin boolean and while loop to try margin value
    
    margin_error = True
    
    while margin_error:
    
        try:
            #Crop the face further with MTCNN
            detector = MTCNN()
    
            #Obtain the first detected face in the cropped face picture
            first_detected_face = detector.detect_faces(image_array)[0]
        
            # get coordinates
            x1, y1, width, height = first_detected_face['box']
            x2, y2 = x1 + width + margin, y1 + height + margin       
            x1 -= margin
            y1 -= margin 
        
            face_array = image_array[y1:y2, x1:x2]
    
        except:
        
            face_array = image_array
        
        try:
            face_array_resized = Image.fromarray(face_array)
            face_array_resized = face_array_resized.resize(dimensions)
            
            margin_error = False
            break
            
        except:
            
            if margin > 0: 
                margin -= 1
            else:
                face_array_resized = Image.fromarray(image_array)
                face_array_resized = face_array_resized.resize(dimensions)
                break
    
    return asarray(face_array_resized)

In [ ]:
resize_picture(photo_df.file_path, dimensions = (160,160), margin = 0)

In [ ]:
photo_df.name.nunique()

In [ ]:
#A search for any name in the dataframe
#photo_df[photo_df['name'].str.contains("veronica", case=False)]

In [ ]:
#photo_df[photo_df['file_path'].str.contains("rm13845", case=False)]

In [ ]:
#A check to see if the file path exists
file_exists = 0
file_not_exist = 0

for i in photo_df.file_path:
    
    if path.exists(i):
        file_exists += 1
    else: 
        file_not_exist += 1
        
print(file_exists)
print(file_not_exist)

In [ ]:
def getImagePixels(image_path):
    return cv2.imread(image_path)
 
photo_df['pixels'] = photo_df['file_path'].apply(getImagePixels)


In [ ]:
photo_df.head()

In [ ]:
def extract_face(filename, required_size=(160, 160)):
    
    # load image from file
    image = Image.open(filename)
    
    # convert to RGB, if needed
    image = image.convert('RGB')
    
    #resize the image
    image = image.resize(required_size)
    
    #Convert to an array
    face_array = asarray(image)
                                                                                
    return face_array

In [ ]:
# load images and extract faces for all images in a directory
def load_faces(directory):
    faces = list()
    # enumerate files
    for filename in listdir(directory):
        # path
        path = directory + filename
        # get face
        face = extract_face(path)
        # store
        faces.append(face)
    return faces

In [ ]:
# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
    X, y = list(), list()
    # enumerate folders, on per class
    for subdir in listdir(directory):
        # path
        path = directory + subdir + '/'
        # skip any files that might be in the dir
        if not isdir(path):
            continue
        # load all faces in the subdirectory
        faces = load_faces(path)
        # create labels
        labels = [subdir for _ in range(len(faces))]
        # summarize progress
        print('>loaded %d examples for class: %s' % (len(faces), subdir)) # store
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)

In [ ]:
a = load_dataset('image_data/imdb_data/')

In [ ]:
a

In [ ]:
# load the facenet model
facenet_model = load_model('FaceNet_Model/facenet_keras.h5')
facenet_model.load_weights('FaceNet_Model/facenet_keras_weights.h5')
print('Loaded Model')

In [ ]:
# Model Summary 
facenet_model.summary()

In [ ]:
def resize_picture(filename, dimensions = (160,160), margin = 0):
            
    # load the image
    image = Image.open(filename)
    image = image.convert('RGB')
    image_array = asarray(image)
    
    #
    face_array_resized = Image.fromarray(face_array)
    face_array_resized = face_array_resized.resize(dimensions)
            
    margin_error = False
            break
            
        except:
            
            if margin > 0: 
                margin -= 1
            else:
                face_array_resized = Image.fromarray(image_array)
                face_array_resized = face_array_resized.resize(dimensions)
                break
    
    return asarray(face_array_resized)

In [ ]:
test = photo_df[:10].copy()

In [ ]:
test.head()

In [ ]:
for _, row in tqdm_notebook(test.iterrows(), 
                         desc="Processing records"):
    # Assuming filename is included in paths column
    # Otherwise, the filename should be added to new_path
    # and the shutil.move
    new_path = os.path.join(
        row['name'] + ' ' + row['dob'],
        os.path.basename(row['file_path'])
    )
    test['file_path2'] = new_path
    
    #shutil.move(row['paths'], new_path)
    

In [ ]:
test.head()

In [ ]:
train, validate, test = np.split(photo_df.sample(frac=1), [int(.6*len(photo_df)), int(.8*len(photo_df))])

In [ ]:
len(test)

In [ ]:
len(validate)

In [ ]:
len(train)

In [ ]:
train

In [ ]:
def resize_picture(filename, dimensions = (160,160), margin = 0):
            
    # load the image
    image = Image.open(filename)
    image = image.convert('RGB')
    image_array = asarray(image)
    
    #Set a margin boolean and while loop to try margin value
    
    margin_error = True
    
    while margin_error:
    
        try:
            #Crop the face further with MTCNN
            detector = MTCNN()
    
            #Obtain the first detected face in the cropped face picture
            first_detected_face = detector.detect_faces(image_array)[0]
        
            # get coordinates
            x1, y1, width, height = first_detected_face['box']
            x2, y2 = x1 + width + margin, y1 + height + margin       
            x1 -= margin
            y1 -= margin 
        
            face_array = image_array[y1:y2, x1:x2]
    
        except:
        
            face_array = image_array
        
        try:
            face_array_resized = Image.fromarray(face_array)
            face_array_resized = face_array_resized.resize(dimensions)
            
            margin_error = False
            break
            
        except:
            
            if margin > 0: 
                margin -= 1
            else:
                face_array_resized = Image.fromarray(image_array)
                face_array_resized = face_array_resized.resize(dimensions)
                break
    
    return asarray(face_array_resized)
    

In [ ]:
photo_df['file_path'][1]

In [ ]:
test = resize_picture(photo_df['file_path'][500], margin = 8)

In [ ]:
test.shape

In [ ]:
plt.imshow(test)

In [ ]:
tqdm.pandas()
photo_df['img_array'] = photo_df['file_path'].progress_apply(lambda x: resize_picture(x, dimensions = (160,160), margin = 8))